In [3]:
import sys, os
sys.path.append(os.path.abspath('/spark-data'))

In [4]:
#Importing Spark Session and important libraries that need to perform things in project
from utils.Data_Ingestion.data_ingestion import load_files 
from utils.Data_Cleaning.data_cleaning import remove_duplicates, validate_emails
from utils.Data_Cleaning.data_cleaning import standardize_date_format, remove_decimal, clean_phone_numbers, handle_negative_values, impute_nulls, drop_nulls, remove_string_from_columns
from utils.save_dataframe_to_files import save_file
from pyspark.sql import SparkSession

In [5]:
#Spark Session Builder that builds the Session in which we work upon

spark = SparkSession.builder \
    .appName("ExceltoDataFrame") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:3.4.1_0.19.0") \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.crealytics#spark-excel_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-367becf4-c82c-4225-801f-ddd5f653b8c4;1.0
	confs: [default]
	found com.crealytics#spark-excel_2.12;3.4.1_0.19.0 in central
	found org.apache.poi#poi;5.2.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.commons#commons-collections4;4.4 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found com.zaxxer#SparseBitSet;1.2 in central
	found org.apache.poi#poi-ooxml;5.2.3 in central
	found org.apache.poi#poi-ooxml-lite;5.2.3 in central
	found org.apache.xmlbeans#xmlbeans;5.1.1 in central
	found com.github.virtuald#curvesapi;1.07 in central
	found com.norbitltd#spoiwo_2.12;2.2.1 in central
	found com.github.tototoshi#scala-csv_2.12;1.3.10 in central
	found com.github.pjfanning#excel-streaming-reader;4.0.5 in central
	found com.gi

In [96]:
dfs = load_files(spark)
for df_name, df in dfs.items():
    print(f"DataFrame Name: {df_name, df}")

2024-09-01 11:22:36,279 - INFO - Successfully processed nested JSON field 'orders' in file: /spark-data/Data_Files/data.json
2024-09-01 11:22:36,297 - INFO - Successfully processed nested JSON field 'reviews' in file: /spark-data/Data_Files/data.json
2024-09-01 11:22:37,225 - INFO - Successfully loaded sheet 'data' from Excel file: /spark-data/Data_Files/data.xlsx
2024-09-01 11:22:37,652 - INFO - Successfully loaded sheet 'product' from Excel file: /spark-data/Data_Files/data.xlsx
2024-09-01 11:22:38,072 - INFO - Successfully loaded sheet 'customer' from Excel file: /spark-data/Data_Files/data.xlsx


DataFrame Name: ('df_data_orders', DataFrame[order_id: bigint, order_item_id: bigint, price: bigint, product_id: bigint, quantity: bigint])
DataFrame Name: ('df_data_reviews', DataFrame[customer_id: bigint, product_id: bigint, rating: bigint, review_date: string, review_id: bigint, review_text: string])
DataFrame Name: ('df_data_data', DataFrame[order_id: double, customer_id: double, order_date: timestamp, total_amount: double, shipping_address: string, billing_address: string])
DataFrame Name: ('df_data_product', DataFrame[product_id: double, product_name: string, category: string, price: double, description: string, inventory: double])
DataFrame Name: ('df_data_customer', DataFrame[customer_id: double, first_name: string, last_name: string, email: string, phone_number: string, city: string, state: string, zip_code: double, birth_date: timestamp])


In [7]:
dfs['df_data_orders'].show()

+--------+-------------+-----+----------+--------+
|order_id|order_item_id|price|product_id|quantity|
+--------+-------------+-----+----------+--------+
|     868|            1|   70|      1571|       7|
|    3151|            2|    8|      3809|      -3|
|    1582|            3|   32|      4896|       8|
|    2578|            4|   64|      2712|       4|
|    2792|            5|   53|      1111|       4|
|    1257|            6|   69|      2874|       1|
|    2389|            7|   40|      2670|       9|
|    2013|            8|   39|      3779|       6|
|    4272|            9|   15|      3109|       5|
|    1260|           10|   21|      2110|       8|
|    3617|           11|   27|      3350|       1|
|    1479|           12|   96|      2917|       9|
|    1405|           13|   61|      1480|       5|
|    4526|           14|   69|      4686|       5|
|    2513|           15|    8|      3146|       8|
|    2346|           16|   92|      4182|       5|
|    3988|           17|   88| 

In [8]:
dfs['df_data_reviews'].show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|       2522|      4663|     3| 2022-12-20|        1|Home create anyon...|
|       2181|      2645|     3| 2024-07-05|        2|Skill herself opt...|
|       3935|      1699|     3| 2024-05-06|        3|Public everyone s...|
|       3453|      1360|     1| 2023-11-05|        4|Level land majori...|
|       2999|      4883|     3| 2024-03-03|        5|Treat fine now di...|
|        863|      3544|     2| 2023-06-02|        6|Land despite buy ...|
|       2348|      1853|     1| 2023-11-11|        7|Ground again goal...|
|       2388|      1652|     3| 2022-11-29|        8|Community materia...|
|       4486|       632|     4| 2023-03-31|        9|Goal after color ...|
|       1379|      2399|     5| 2024-02-11|       10|Beyond coach meet...|
|        309|      3241| 

In [9]:
dfs['df_data_data'].show()

+--------+-----------+-------------------+------------+--------------------+--------------------+
|order_id|customer_id|         order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+-------------------+------------+--------------------+--------------------+
|     1.0|      462.0|2019-08-26 00:00:00|      4007.0|60585 Moore Overp...|775 Roberson Knol...|
|     2.0|     2820.0|2021-04-22 00:00:00|      7240.0|3278 Price Isle\n...|2055 Curry Street...|
|     3.0|     1934.0|2024-02-21 00:00:00|      8318.0|Unit 8425 Box 079...|USNV Strong\nFPO ...|
|     4.0|     2694.0|2021-01-14 00:00:00|      4992.0|5566 Brittany Sta...|524 Allen Shoal S...|
|     5.0|      134.0|2020-03-14 00:00:00|      5385.0|4289 Michael Esta...|38637 Cheryl Junc...|
|     6.0|     4651.0|2020-12-30 00:00:00|      3512.0|83184 Shelly Land...|80329 Dawn Road S...|
|     7.0|     1696.0|2023-05-30 00:00:00|      4592.0|7127 Arroyo Land\...|787 Smith Islands...|
|     8.0|     1969.

In [10]:
dfs['df_data_product'].show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|       1.0|          bag|       no|448.0|Summer now health...|    655.0|
|       2.0|        happy|   answer|844.0|Employee discuss ...|    230.0|
|       3.0|         same|      his|681.0|Million mother pu...|    831.0|
|       4.0|      prevent|   around| 34.0|Because under mea...|    804.0|
|       5.0|        quite|   summer|663.0|Approach Mr task ...|    351.0|
|       6.0|        which|  purpose|386.0|Leave charge mode...|     10.0|
|       7.0|       beyond|   beyond|879.0|Shoulder power in...|    282.0|
|       8.0|     although|     home|650.0|Exactly continue ...|    818.0|
|       9.0|environmental| although|  7.0|Really admit kind...|    128.0|
|      10.0|       assume|professor|945.0|Bar truth here gr...|    168.0|
|      11.0|      usually|   figure|43

### Cleaning df_customer

In [11]:
df_customer = dfs['df_data_customer']

In [12]:
df_customer.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|         birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|        1.0|    Crystal|  Edwards|mcmahonemily@exam...|        511.782.4381|    Fergusonville|         Iowa| 91344.0|1995-02-28 00:00:00|
|        2.0|   Jennifer|    Jones|douglasedwards@ex...|          9187202495|     Jenniferport|   New Jersey| 59371.0|1951-01-27 00:00:00|
|        3.0|  Catherine|    Crane|douglaskevin@exam...|  (381)601-2272x7972|  Jenniferchester|West Virginia| 49812.0|1966-10-07 00:00:00|
|        4.0|     Lauren|    Irwin|aguirresteven@exa...|    001-707-942-4829|      New Melissa|     Michigan| 17279.0|1936-07-28 00:00:00|
|        5.0|Christopher|  

In [13]:
#This is for when you want to apply this remove_duplicate on specific column of dataframe but it is optional
# df_customer_demo = remove_duplicates(df_customer, columns=["customer_id"]).show()

#For remove Duplicates on df_customer dataframe
df_customer_remove_duplicates = remove_duplicates(df_customer)

2024-09-01 09:43:29,177 - INFO - Successfully removed duplicate rows based on all columns.


In [14]:
df_customer_handle_nulls = drop_nulls(df_customer, 'all', subset=["customer_id"])
df_customer_handle_nulls = impute_nulls(df_customer_handle_nulls, columns=["first_name", "last_name", "city", "state", "zip_code", "birth_date"], method="constant",value="Not Available")

2024-09-01 09:43:29,485 - INFO - Dropped rows/columns with null values using method 'all'.
2024-09-01 09:43:29,636 - INFO - Imputed nulls in column 'first_name' with constant value: Not Available
2024-09-01 09:43:29,649 - INFO - Imputed nulls in column 'last_name' with constant value: Not Available
2024-09-01 09:43:29,663 - INFO - Imputed nulls in column 'city' with constant value: Not Available
2024-09-01 09:43:29,679 - INFO - Imputed nulls in column 'state' with constant value: Not Available
2024-09-01 09:43:29,698 - INFO - Imputed nulls in column 'zip_code' with constant value: Not Available
2024-09-01 09:43:29,712 - INFO - Imputed nulls in column 'birth_date' with constant value: Not Available


In [15]:
df_customer_handle_nulls.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|         birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+-------------------+
|        1.0|    Crystal|  Edwards|mcmahonemily@exam...|        511.782.4381|    Fergusonville|         Iowa| 91344.0|1995-02-28 00:00:00|
|        2.0|   Jennifer|    Jones|douglasedwards@ex...|          9187202495|     Jenniferport|   New Jersey| 59371.0|1951-01-27 00:00:00|
|        3.0|  Catherine|    Crane|douglaskevin@exam...|  (381)601-2272x7972|  Jenniferchester|West Virginia| 49812.0|1966-10-07 00:00:00|
|        4.0|     Lauren|    Irwin|aguirresteven@exa...|    001-707-942-4829|      New Melissa|     Michigan| 17279.0|1936-07-28 00:00:00|
|        5.0|Christopher|  

In [16]:
df_customer_correct_birthdate = standardize_date_format(df_customer_handle_nulls, 'birth_date')


2024-09-01 09:43:30,595 - INFO - Successfully standardized date format in column 'birth_date' to 'yyyy-MM-dd'.


In [17]:
df_customer_correct_birthdate.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|        1.0|    Crystal|  Edwards|mcmahonemily@exam...|        511.782.4381|    Fergusonville|         Iowa| 91344.0|1995-02-28|
|        2.0|   Jennifer|    Jones|douglasedwards@ex...|          9187202495|     Jenniferport|   New Jersey| 59371.0|1951-01-27|
|        3.0|  Catherine|    Crane|douglaskevin@exam...|  (381)601-2272x7972|  Jenniferchester|West Virginia| 49812.0|1966-10-07|
|        4.0|     Lauren|    Irwin|aguirresteven@exa...|    001-707-942-4829|      New Melissa|     Michigan| 17279.0|1936-07-28|
|        5.0|Christopher|   Nelson|dustin50@example.org|  735.814.4145x28586|       Hannah

In [18]:
df_customer_remove_decimal = remove_decimal(df_customer_correct_birthdate, columns=["zip_code", "customer_id"])


2024-09-01 09:43:31,370 - INFO - Successfully removed decimal values from column 'zip_code'.
2024-09-01 09:43:31,392 - INFO - Successfully removed decimal values from column 'customer_id'.


In [19]:
df_customer_remove_decimal.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|        511.782.4381|    Fergusonville|         Iowa|   91344|1995-02-28|
|          2|   Jennifer|    Jones|douglasedwards@ex...|          9187202495|     Jenniferport|   New Jersey|   59371|1951-01-27|
|          3|  Catherine|    Crane|douglaskevin@exam...|  (381)601-2272x7972|  Jenniferchester|West Virginia|   49812|1966-10-07|
|          4|     Lauren|    Irwin|aguirresteven@exa...|    001-707-942-4829|      New Melissa|     Michigan|   17279|1936-07-28|
|          5|Christopher|   Nelson|dustin50@example.org|  735.814.4145x28586|       Hannah

In [20]:
df_customer_clean_phone_numbers = clean_phone_numbers(df_customer_remove_decimal, 'phone_number')

2024-09-01 09:43:32,233 - INFO - Phone numbers in column 'phone_number' cleaned successfully.


In [21]:
df_customer_clean_phone_numbers.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|   +1 (511) 782-4381|    Fergusonville|         Iowa|   91344|1995-02-28|
|          2|   Jennifer|    Jones|douglasedwards@ex...|   +1 (918) 720-2495|     Jenniferport|   New Jersey|   59371|1951-01-27|
|          3|  Catherine|    Crane|douglaskevin@exam...|+1 (381) 601-2272...|  Jenniferchester|West Virginia|   49812|1966-10-07|
|          4|     Lauren|    Irwin|aguirresteven@exa...|Invalid phone number|      New Melissa|     Michigan|   17279|1936-07-28|
|          5|Christopher|   Nelson|dustin50@example.org|+1 (735) 814-4145...|       Hannah

In [22]:
df_customer_invalid_email_handel = validate_emails(df_customer_clean_phone_numbers, "email", "Invalid Email")

2024-09-01 09:43:33,985 - INFO - Successfully validated email addresses in column 'email'.


In [23]:
df_customer_cleaned = df_customer_invalid_email_handel
df_customer_cleaned.show()

+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|customer_id| first_name|last_name|               email|        phone_number|             city|        state|zip_code|birth_date|
+-----------+-----------+---------+--------------------+--------------------+-----------------+-------------+--------+----------+
|          1|    Crystal|  Edwards|mcmahonemily@exam...|   +1 (511) 782-4381|    Fergusonville|         Iowa|   91344|1995-02-28|
|          2|   Jennifer|    Jones|douglasedwards@ex...|   +1 (918) 720-2495|     Jenniferport|   New Jersey|   59371|1951-01-27|
|          3|  Catherine|    Crane|douglaskevin@exam...|+1 (381) 601-2272...|  Jenniferchester|West Virginia|   49812|1966-10-07|
|          4|     Lauren|    Irwin|aguirresteven@exa...|Invalid phone number|      New Melissa|     Michigan|   17279|1936-07-28|
|          5|Christopher|   Nelson|dustin50@example.org|+1 (735) 814-4145...|       Hannah

In [24]:
save_file(df_customer_cleaned, "/spark-data/Cleaned_Csv_File", "df_customer_cleaned")

2024-09-01 09:43:35,092 - INFO - Saving DataFrame to temporary directory...
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleaning phone number 'None': expected string or bytes-like object, got 'NoneType'
ERROR:root:Error cleanin

'/spark-data/Cleaned_Csv_File/df_customer_cleaned.csv'

### Cleanining df_data_product table

In [25]:
df_product = dfs['df_data_product']

In [26]:
df_product.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|       1.0|          bag|       no|448.0|Summer now health...|    655.0|
|       2.0|        happy|   answer|844.0|Employee discuss ...|    230.0|
|       3.0|         same|      his|681.0|Million mother pu...|    831.0|
|       4.0|      prevent|   around| 34.0|Because under mea...|    804.0|
|       5.0|        quite|   summer|663.0|Approach Mr task ...|    351.0|
|       6.0|        which|  purpose|386.0|Leave charge mode...|     10.0|
|       7.0|       beyond|   beyond|879.0|Shoulder power in...|    282.0|
|       8.0|     although|     home|650.0|Exactly continue ...|    818.0|
|       9.0|environmental| although|  7.0|Really admit kind...|    128.0|
|      10.0|       assume|professor|945.0|Bar truth here gr...|    168.0|
|      11.0|      usually|   figure|43

In [27]:
df_product_remove_duplicate = remove_duplicates(df_product)

2024-09-01 09:43:39,499 - INFO - Successfully removed duplicate rows based on all columns.


In [28]:
df_product_remove_decimals = remove_decimal(df_product, columns=["product_id", "inventory"])

2024-09-01 09:43:39,528 - INFO - Successfully removed decimal values from column 'product_id'.


2024-09-01 09:43:39,545 - INFO - Successfully removed decimal values from column 'inventory'.


In [29]:
df_product_remove_decimals.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [30]:
df_product_handling_negative_val = handle_negative_values(df_product_remove_decimals, columns=["price"], operation="absolute")

2024-09-01 09:43:40,159 - INFO - Successfully handled negative values in column 'price' by converting to absolute values.


In [31]:
df_product_handling_negative_val.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [97]:
df_product_handling_negative_val = handle_negative_values(df_product_handling_negative_val, columns=["inventory"], operation='zero')

2024-09-01 11:23:32,650 - INFO - Successfully handled negative values in column 'inventory' by setting them to zero.


In [33]:
df_product_handle_nulls = drop_nulls(df_product_handling_negative_val, "all", subset=["product_id", "product_name"])
df_product_handle_nulls = impute_nulls(df_product_handle_nulls, columns=["category", "description"], method="constant", value="Not Available")
df_product_handle_nulls = impute_nulls(df_product_handle_nulls, columns=["price"], method="mode")

2024-09-01 09:43:40,945 - INFO - Dropped rows/columns with null values using method 'all'.
2024-09-01 09:43:40,989 - INFO - Imputed nulls in column 'category' with constant value: Not Available
2024-09-01 09:43:40,999 - INFO - Imputed nulls in column 'description' with constant value: Not Available
2024-09-01 09:43:42,554 - INFO - Imputed nulls in column 'price' with mode value: 637.0


In [37]:
df_product_desciption_filter = remove_string_from_columns(df_product_handle_nulls, columns=["description"], string_to_remove="\n")

2024-09-01 09:45:08,357 - INFO - Successfully removed '
' from column 'description'.


In [38]:
df_cleaned_product = df_product_desciption_filter

In [39]:
df_cleaned_product.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|         1|          bag|       no|448.0|Summer now health...|      655|
|         2|        happy|   answer|844.0|Employee discuss ...|      230|
|         3|         same|      his|681.0|Million mother pu...|      831|
|         4|      prevent|   around| 34.0|Because under mea...|      804|
|         5|        quite|   summer|663.0|Approach Mr task ...|      351|
|         6|        which|  purpose|386.0|Leave charge mode...|       10|
|         7|       beyond|   beyond|879.0|Shoulder power in...|      282|
|         8|     although|     home|650.0|Exactly continue ...|      818|
|         9|environmental| although|  7.0|Really admit kind...|      128|
|        10|       assume|professor|945.0|Bar truth here gr...|      168|
|        11|      usually|   figure|43

In [40]:
df_cleaned_product.select("description").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                                                                                            |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Summer now health each budget me. Before hard later nice for. Meet because door worry. Too be pick effect he wide. Network executive fight somebody discuss science under.                             |
|Employee discuss society fly watch. Exactly affect seven physical big artist there. Into event still region thus left. Will tree box political popular force. Budget hold such as mind individu

In [41]:
save_file(df_cleaned_product, "/spark-data/Cleaned_Csv_File", "df_product_cleaned")

2024-09-01 09:45:23,615 - INFO - Saving DataFrame to temporary directory...
24/09/01 09:45:24 WARN TaskSetManager: Stage 23 contains a task of very large size (1070 KiB). The maximum recommended task size is 1000 KiB.
2024-09-01 09:45:24,414 - INFO - DataFrame saved successfully to temporary directory.
2024-09-01 09:45:24,417 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_product_cleaned.csv
2024-09-01 09:45:24,419 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_product_cleaned.csv'

### df_data_data table cleaning


In [42]:
df_data = dfs['df_data_data']

In [43]:
df_data.show()

+--------+-----------+-------------------+------------+--------------------+--------------------+
|order_id|customer_id|         order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+-------------------+------------+--------------------+--------------------+
|     1.0|      462.0|2019-08-26 00:00:00|      4007.0|60585 Moore Overp...|775 Roberson Knol...|
|     2.0|     2820.0|2021-04-22 00:00:00|      7240.0|3278 Price Isle\n...|2055 Curry Street...|
|     3.0|     1934.0|2024-02-21 00:00:00|      8318.0|Unit 8425 Box 079...|USNV Strong\nFPO ...|
|     4.0|     2694.0|2021-01-14 00:00:00|      4992.0|5566 Brittany Sta...|524 Allen Shoal S...|
|     5.0|      134.0|2020-03-14 00:00:00|      5385.0|4289 Michael Esta...|38637 Cheryl Junc...|
|     6.0|     4651.0|2020-12-30 00:00:00|      3512.0|83184 Shelly Land...|80329 Dawn Road S...|
|     7.0|     1696.0|2023-05-30 00:00:00|      4592.0|7127 Arroyo Land\...|787 Smith Islands...|
|     8.0|     1969.

In [44]:
df_data_removed_duplicate = remove_duplicates(df_data)

2024-09-01 09:45:36,547 - INFO - Successfully removed duplicate rows based on all columns.


In [45]:
df_data_handle_nulls = drop_nulls(df_data_removed_duplicate, "all", subset=["order_id", "customer_id"])
df_data_handle_nulls = impute_nulls(df_data_handle_nulls, columns=["order_date", "shipping_address", "billing_address"], method="constant",value="Not Available")
df_data_handle_nulls = impute_nulls(df_data_handle_nulls, columns=["total_amount"], method="mean")


2024-09-01 09:45:38,623 - INFO - Dropped rows/columns with null values using method 'all'.
2024-09-01 09:45:38,684 - INFO - Imputed nulls in column 'order_date' with constant value: Not Available
2024-09-01 09:45:38,692 - INFO - Imputed nulls in column 'shipping_address' with constant value: Not Available
2024-09-01 09:45:38,706 - INFO - Imputed nulls in column 'billing_address' with constant value: Not Available
2024-09-01 09:45:39,823 - INFO - Imputed nulls in column 'total_amount' with mean value: 4976.722


In [46]:
df_data_handle_nulls.show()

+--------+-----------+-------------------+------------+--------------------+--------------------+
|order_id|customer_id|         order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+-------------------+------------+--------------------+--------------------+
|   131.0|     4958.0|2020-05-10 00:00:00|       569.0|7886 Higgins Moto...|14788 Robert Land...|
|   297.0|       46.0|2020-03-11 00:00:00|      1331.0|PSC 4470, Box 299...|16387 Novak Shoal...|
|   330.0|     1082.0|2025-05-03 00:00:00|      4010.0|758 Sarah Plaza\n...|938 Brian Road\nL...|
|    62.0|     2934.0|2024-06-01 00:00:00|      1727.0|7971 Vicki Key Ap...|465 Kramer Way Ap...|
|   266.0|     3929.0|2022-05-30 00:00:00|       680.0|57853 Chen Way\nL...|055 Butler Extens...|
|    43.0|      118.0|2021-09-26 00:00:00|      4884.0|9822 Anne Trace A...|1511 Hill Keys Ap...|
|    95.0|     4222.0|2023-08-27 00:00:00|      3765.0|Unit 4050 Box 161...|06811 Sean Crossi...|
|   167.0|     3800.

In [47]:
df_data_date_formated = standardize_date_format(df_data_handle_nulls, "order_date")

2024-09-01 09:45:44,125 - INFO - Successfully standardized date format in column 'order_date' to 'yyyy-MM-dd'.


In [48]:
df_data_remove_decimals = remove_decimal(df_data_date_formated, columns=['order_id', 'customer_id', 'total_amount'])

2024-09-01 09:45:46,298 - INFO - Successfully removed decimal values from column 'order_id'.


2024-09-01 09:45:46,314 - INFO - Successfully removed decimal values from column 'customer_id'.
2024-09-01 09:45:46,332 - INFO - Successfully removed decimal values from column 'total_amount'.


In [49]:
df_data_negative_handel = handle_negative_values(df_data_remove_decimals, columns=["order_id", "customer_id", "total_amount"], operation="absolute")

2024-09-01 09:45:49,352 - INFO - Successfully handled negative values in column 'order_id' by converting to absolute values.


2024-09-01 09:45:49,375 - INFO - Successfully handled negative values in column 'customer_id' by converting to absolute values.
2024-09-01 09:45:49,396 - INFO - Successfully handled negative values in column 'total_amount' by converting to absolute values.


In [53]:
df_handel_address = remove_string_from_columns(df_data_negative_handel, columns=["shipping_address", "billing_address"], string_to_remove="\n")

2024-09-01 09:47:07,951 - INFO - Successfully removed '
' from column 'shipping_address'.


2024-09-01 09:47:07,968 - INFO - Successfully removed '
' from column 'billing_address'.


In [69]:
df_order_cleaned = df_handel_address

In [70]:
df_order_cleaned.show()

+--------+-----------+----------+------------+--------------------+--------------------+
|order_id|customer_id|order_date|total_amount|    shipping_address|     billing_address|
+--------+-----------+----------+------------+--------------------+--------------------+
|     131|       4958|2020-05-10|         569|7886 Higgins Moto...|14788 Robert Land...|
|     297|         46|2020-03-11|        1331|PSC 4470, Box 299...|16387 Novak Shoal...|
|     330|       1082|2025-05-03|        4010|758 Sarah Plaza P...|938 Brian Road La...|
|      62|       2934|2024-06-01|        1727|7971 Vicki Key Ap...|465 Kramer Way Ap...|
|     266|       3929|2022-05-30|         680|57853 Chen Way La...|055 Butler Extens...|
|      43|        118|2021-09-26|        4884|9822 Anne Trace A...|1511 Hill Keys Ap...|
|      95|       4222|2023-08-27|        3765|Unit 4050 Box 161...|06811 Sean Crossi...|
|     167|       3800|2022-12-09|        6259|16060 Mason Creek...|196 Joshua Meadow...|
|     300|       1337

In [91]:
save_file(df_order_cleaned, "/spark-data/Cleaned_Csv_File","df_order_cleaned")

2024-09-01 11:10:39,524 - INFO - Saving DataFrame to temporary directory...
2024-09-01 11:10:40,338 - INFO - DataFrame saved successfully to temporary directory.
2024-09-01 11:10:40,339 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_order_cleaned.csv
2024-09-01 11:10:40,340 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_order_cleaned.csv'

### df_data_reviews cleaning

In [57]:
df_review = dfs["df_data_reviews"]

In [58]:
df_review.show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|       2522|      4663|     3| 2022-12-20|        1|Home create anyon...|
|       2181|      2645|     3| 2024-07-05|        2|Skill herself opt...|
|       3935|      1699|     3| 2024-05-06|        3|Public everyone s...|
|       3453|      1360|     1| 2023-11-05|        4|Level land majori...|
|       2999|      4883|     3| 2024-03-03|        5|Treat fine now di...|
|        863|      3544|     2| 2023-06-02|        6|Land despite buy ...|
|       2348|      1853|     1| 2023-11-11|        7|Ground again goal...|
|       2388|      1652|     3| 2022-11-29|        8|Community materia...|
|       4486|       632|     4| 2023-03-31|        9|Goal after color ...|
|       1379|      2399|     5| 2024-02-11|       10|Beyond coach meet...|
|        309|      3241| 

In [60]:
df_review_remove_duplicate = remove_duplicates(df_review)

2024-09-01 10:02:05,865 - INFO - Successfully removed duplicate rows based on all columns.


In [61]:
df_review_drop_nulls = drop_nulls(df_review_remove_duplicate, "all", subset=["product_id", "rating"])
df_review_impute_nulls = impute_nulls(df_review_drop_nulls, columns=["customer_id", "review_date", "review_id", "review_text"], method="constant", value="Not Available")

2024-09-01 10:11:51,486 - INFO - Dropped rows/columns with null values using method 'all'.
2024-09-01 10:11:51,552 - INFO - Imputed nulls in column 'customer_id' with constant value: Not Available
2024-09-01 10:11:51,561 - INFO - Imputed nulls in column 'review_date' with constant value: Not Available
2024-09-01 10:11:51,572 - INFO - Imputed nulls in column 'review_id' with constant value: Not Available
2024-09-01 10:11:51,581 - INFO - Imputed nulls in column 'review_text' with constant value: Not Available


In [62]:
df_review_review_handel = remove_string_from_columns(df_review_impute_nulls, columns=["review_text"], string_to_remove="\n")

2024-09-01 10:13:07,095 - INFO - Successfully removed '
' from column 'review_text'.


In [63]:
df_review_review_handel.show()

+-----------+----------+------+-----------+---------+--------------------+
|customer_id|product_id|rating|review_date|review_id|         review_text|
+-----------+----------+------+-----------+---------+--------------------+
|        166|      3108|     1| 2024-02-12|      363|Realize option ap...|
|       1662|      3924|     5| 2024-05-21|      493|Goal rich travel ...|
|       4238|       281|     4| 2023-01-01|      619|Fish future kind ...|
|        520|       498|     2| 2023-07-03|      678|Computer itself t...|
|         58|      1360|     1| 2024-02-01|     1290|Into upon enough ...|
|       2277|      3726|     3| 2023-12-11|     1580|Word peace race m...|
|       3290|      2684|     5| 2024-06-20|     1627|Market process pr...|
|       4476|      2171|     1| 2023-07-07|     1939|South song race p...|
|         63|      4951|     2| 2022-12-29|     2547|Tv character seni...|
|         84|       983|     2| 2024-03-12|     2572|Six the admit bes...|
|        572|       416| 

In [64]:
df_review_review_date_handel = standardize_date_format(df_review_review_handel, "review_date")

2024-09-01 10:14:12,554 - INFO - Successfully standardized date format in column 'review_date' to 'yyyy-MM-dd'.


In [65]:
df_review_cleaned = df_review_review_date_handel

In [66]:
save_file(df_review_cleaned, "/spark-data/Cleaned_Csv_File", "df_review_cleaned")

2024-09-01 10:14:29,869 - INFO - Saving DataFrame to temporary directory...
2024-09-01 10:14:30,744 - INFO - DataFrame saved successfully to temporary directory.
2024-09-01 10:14:30,748 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_review_cleaned.csv
2024-09-01 10:14:30,750 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_review_cleaned.csv'

### Cleaning df_orders

In [77]:
df_order_items = dfs["df_data_orders"]

In [78]:
df_order_items.show()

+--------+-------------+-----+----------+--------+
|order_id|order_item_id|price|product_id|quantity|
+--------+-------------+-----+----------+--------+
|     868|            1|   70|      1571|       7|
|    3151|            2|    8|      3809|      -3|
|    1582|            3|   32|      4896|       8|
|    2578|            4|   64|      2712|       4|
|    2792|            5|   53|      1111|       4|
|    1257|            6|   69|      2874|       1|
|    2389|            7|   40|      2670|       9|
|    2013|            8|   39|      3779|       6|
|    4272|            9|   15|      3109|       5|
|    1260|           10|   21|      2110|       8|
|    3617|           11|   27|      3350|       1|
|    1479|           12|   96|      2917|       9|
|    1405|           13|   61|      1480|       5|
|    4526|           14|   69|      4686|       5|
|    2513|           15|    8|      3146|       8|
|    2346|           16|   92|      4182|       5|
|    3988|           17|   88| 

In [76]:
df_order_remove_duplicates = remove_duplicates(df_order_items)

2024-09-01 10:17:45,598 - INFO - Successfully removed duplicate rows based on all columns.


In [94]:
df_order_neg_handel = handle_negative_values(df_order_remove_duplicates, columns=["quantity"], operation="zero")

2024-09-01 11:19:32,546 - INFO - Successfully handled negative values in column 'quantity' by setting them to zero.


In [95]:
df_order_neg_handel = handle_negative_values(df_order_neg_handel, columns=["order_id", "order_item_id", "price", "product_id"], operation="absolute")

2024-09-01 11:22:09,177 - INFO - Successfully handled negative values in column 'order_id' by converting to absolute values.


2024-09-01 11:22:09,198 - INFO - Successfully handled negative values in column 'order_item_id' by converting to absolute values.
2024-09-01 11:22:09,220 - INFO - Successfully handled negative values in column 'price' by converting to absolute values.
2024-09-01 11:22:09,239 - INFO - Successfully handled negative values in column 'product_id' by converting to absolute values.


In [87]:
df_order_items_cleaned = df_order_neg_handel

In [90]:
save_file(df_order_items_cleaned, "/spark-data/Cleaned_Csv_File", "df_order_item_cleaned")

2024-09-01 11:10:13,364 - INFO - Saving DataFrame to temporary directory...


2024-09-01 11:10:13,661 - INFO - DataFrame saved successfully to temporary directory.
2024-09-01 11:10:13,664 - INFO - File moved to final output path: /spark-data/Cleaned_Csv_File/df_order_item_cleaned.csv
2024-09-01 11:10:13,666 - INFO - Temporary directory removed: /spark-data/Cleaned_Csv_File/temp_dir


'/spark-data/Cleaned_Csv_File/df_order_item_cleaned.csv'

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import count, col

def profile_data(df: DataFrame, profile_name: str) -> dict:
    """
    Profiles the data by calculating basic metrics like row count, distinct count, and null count.
    
    :param df: DataFrame to profile.
    :param profile_name: Name of the profile (e.g., 'Before Cleaning', 'After Cleaning').
    :return: Dictionary containing the profile metrics.
    """
    profile = {}
    profile['profile_name'] = profile_name
    profile['row_count'] = df.count()
    
    for column in df.columns:
        profile[f'{column}_distinct'] = df.select(column).distinct().count()
        profile[f'{column}_null_count'] = df.filter(col(column).isNull()).count()
    
    return profile

def generate_quality_report(before_profile: dict, after_profile: dict):
    """
    Generates a quality report by comparing the before and after profiles.
    
    :param before_profile: Dictionary containing the profile metrics before cleaning.
    :param after_profile: Dictionary containing the profile metrics after cleaning.
    :return: A dictionary summarizing the changes.
    """
    report = {}
    report['profile_name'] = f"Comparison Report: {before_profile['profile_name']} vs {after_profile['profile_name']}"
    
    for key in before_profile:
        if key != 'profile_name':
            before_value = before_profile[key]
            after_value = after_profile[key]
            report[key] = {
                'before': before_value,
                'after': after_value,
                'difference': before_value - after_value
            }
    
    return report

# Example usage:
before_profile = profile_data(dfs["df_data_product"], "Before Cleaning")

demo = impute_nulls(dfs["df_data_product"], "description", method="constant", value="Not Available")
# Apply your data cleaning functions here
after_profile = profile_data(demo, "After Cleaning")

quality_report = generate_quality_report(before_profile, after_profile)

# Convert report to a DataFrame for better readability
report_df = spark.createDataFrame([
    (k, v['before'], v['after'], v['difference']) for k, v in quality_report.items() if k != 'profile_name'
], ['Metric', 'Before', 'After', 'Difference'])

report_df.show()


2024-09-01 09:18:52,066 - ERROR - Error imputing nulls in columns 'description' using method 'constant': [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `d` cannot be resolved. Did you mean one of the following? [`price`, `category`, `inventory`, `product_id`, `description`].;
'Project [product_id#47, product_name#48, category#49, price#50, description#51, inventory#52, CASE WHEN (isnull('d) OR ('d = null)) THEN null ELSE 'd END AS d#1573]
+- Relation [product_id#47,product_name#48,category#49,price#50,description#51,inventory#52] ExcelRelation(com.crealytics.spark.excel.CellRangeAddressDataLocator@542594aa,true,false,false,true,false,false,None,None,10,com.crealytics.spark.excel.DefaultWorkbookReader@2b4410ca)



+--------------------+------+-----+----------+
|              Metric|Before|After|Difference|
+--------------------+------+-----+----------+
|           row_count|  5000| 5000|         0|
| product_id_distinct|  5000| 5000|         0|
|product_id_null_c...|     0|    0|         0|
|product_name_dist...|   967|  967|         0|
|product_name_null...|     0|    0|         0|
|   category_distinct|   967|  967|         0|
| category_null_count|     0|    0|         0|
|      price_distinct|  1068| 1068|         0|
|    price_null_count|     0|    0|         0|
|description_distinct|  4958| 4958|         0|
|description_null_...|     0|    0|         0|
|  inventory_distinct|   996|  996|         0|
|inventory_null_count|     0|    0|         0|
+--------------------+------+-----+----------+



In [ ]:
demo.show()

+----------+-------------+---------+-----+--------------------+---------+
|product_id| product_name| category|price|         description|inventory|
+----------+-------------+---------+-----+--------------------+---------+
|       1.0|          bag|       no|448.0|Summer now health...|    655.0|
|       2.0|        happy|   answer|844.0|Employee discuss ...|    230.0|
|       3.0|         same|      his|681.0|Million mother pu...|    831.0|
|       4.0|      prevent|   around| 34.0|Because under mea...|    804.0|
|       5.0|        quite|   summer|663.0|Approach Mr task ...|    351.0|
|       6.0|        which|  purpose|386.0|Leave charge mode...|     10.0|
|       7.0|       beyond|   beyond|879.0|Shoulder power in...|    282.0|
|       8.0|     although|     home|650.0|Exactly continue ...|    818.0|
|       9.0|environmental| although|  7.0|Really admit kind...|    128.0|
|      10.0|       assume|professor|945.0|Bar truth here gr...|    168.0|
|      11.0|      usually|   figure|43